# **Product Recognition of Books**

## Image Processing and Computer Vision - Assignment Module #1

---

## Approach Overview

This solution implements a **traditional computer vision pipeline** for detecting books on shelves.

### Key Design Decisions

After extensive experimentation, we adopted a **simplified pipeline** based on the following observations about our dataset:

1. **Books are nearly planar**: Shelf images have minimal perspective distortion
2. **Books are upright or horizontal**: Spines can be vertical or horizontal (stacked shelves)
3. **Scale is consistent**: Model and scene images have similar scale
4. **Multiple copies are adjacent**: Same book appears side-by-side on shelves

These characteristics led us to choose:

| Component | Choice | Justification |
|-----------|--------|---------------|
| Features | RootSIFT | ~15-30% better matching than standard SIFT |
| Matching | **Combined 5NN + BF ratio test** | Maximizes match recall for low-texture models |
| Geometric model | **Similarity transform** (4 DOF) | Uniform scale + rotation + translation; strongest inductive bias for books |
| Multi-instance | Iterative masking + **Orientation-aware template verification** | Handles both textured and low-texture books in any orientation |

### Why Similarity Transform (4 DOF) Instead of Full Affine (6 DOF) or Homography (8 DOF)?

| Property | Homography | Full Affine | Similarity (Partial Affine) |
|----------|------------|-------------|----------------------------|
| Degrees of freedom | 8 | 6 | **4** |
| Minimum points | 4 | 3 | **2** |
| Handles perspective | Yes | No | No |
| Preserves aspect ratio | No | No | **Yes** |
| Allows shear | Yes | Yes | **No** |
| Stability with few points | Lowest | Medium | **Highest** |

Books on shelves do not undergo perspective distortion, shear, or non-uniform scaling. The similarity transform (`cv2.estimateAffinePartial2D`) encodes exactly this **inductive bias**: it only allows uniform scaling, rotation, and translation. This makes RANSAC converge faster and more reliably, especially with the limited matches (~8-15) typical of narrow book spines.

The resulting 2×3 matrix is converted to a 3×3 homography matrix (by appending `[0, 0, 1]`) for convenient corner projection via `cv2.perspectiveTransform`.

### Two-Phase Multi-Instance Detection

Some book spines (especially narrow ones with limited texture) yield too few feature matches to detect all copies via iterative RANSAC alone. To address this, we employ a **two-phase** strategy:

1. **Phase 1 (Feature-based)**: Standard iterative similarity estimation with keypoint masking
2. **Phase 2 (Template-based)**: If Phase 1 found at least one instance, extract the detected scene region as a template and use **Normalized Cross-Correlation (NCC)** to search for additional copies. The search direction is **orientation-aware**: it follows the axis along which copies are expected to be arranged (horizontal for vertical books, vertical for horizontal/stacked books).

This combines the geometric robustness of feature matching with the appearance-level sensitivity of template matching.

## 1. Setup and Imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Tuple, Optional, Set
from dataclasses import dataclass
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print(f"OpenCV version: {cv2.__version__}")

OpenCV version: 4.13.0


## 2. Configuration

All parameters are centralized here with justifications.

In [2]:
class Config:
    """
    Configuration parameters for the detection pipeline.

    Each parameter is justified based on dataset characteristics
    and experimental results.
    """

    # === PREPROCESSING ===
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    # Normalizes lighting variations across shelf images
    CLAHE_CLIP_LIMIT = 2.0   # Moderate contrast enhancement
    CLAHE_GRID_SIZE = (4, 4) # 4x4 provides finer local adaptation than 8x8
    CLAHE_FLAG = True  # able or disable
    # Justification: Experiments showed fixed CLAHE outperformed adaptive selection

    # === FEATURE DETECTION ===
    # SIFT parameters (used via RootSIFT)
    SIFT_FEATURES = 0                # 0 = detect all features
    SIFT_CONTRAST_THRESHOLD = 0.01   # Lower than default 0.04 for more keypoints
    SIFT_EDGE_THRESHOLD = 15         # Slightly higher than default 10

    # === FEATURE MATCHING ===
    # Three complementary matching strategies merged via union:
    #
    # 1) 5NN via FLANN: allows one model keypoint to match multiple scene
    #    locations, essential for multi-instance detection.
    # 2) BF 2NN ratio test: recovers matches that 5NN's group filtering misses.
    # 3) BF 5NN consecutive ratio test (inspired by Conti & Preda):
    #    for each model keypoint's 5 neighbors, keeps match[i] if
    #    match[i].distance < ratio * match[i+1].distance. This captures
    #    good matches at any rank, not just the best.
    DISTANCE_THRESHOLD = 0.8  # 5NN: relative quality threshold
    OUTLIER_RATIO = 1.5       # 5NN: filter matches far from the group
    BF_RATIO = 0.85           # BF 2NN: Lowe's ratio test threshold
    KNN_CONSECUTIVE_RATIO = 0.6  # BF 5NN: consecutive neighbor ratio test
                                 # From reference implementation; stricter
                                 # threshold compensated by checking all ranks

    # === GEOMETRIC VERIFICATION ===
    # Similarity transformation (4 DOF) with RANSAC
    # Justification: Books are planar, so we inject inductive bias
    # using estimateAffinePartial2D (uniform scale + rotation + translation).
    # This is more constrained than full affine (6 DOF) or homography (8 DOF),
    # providing more stable estimation with limited correspondences.
    MIN_MATCH_COUNT = 5            # Minimum matches to attempt estimation
    RANSAC_REPROJ_THRESHOLD = 3.5  # Pixels — allows small localization errors

    # === DETECTION VALIDATION ===
    MIN_INLIERS = 5            # Minimum inliers for valid detection
    MIN_INLIERS_RATIO = 0.25   # At least 25% of matches should be inliers
    MIN_AREA = 1000            # Minimum bounding box area (pixels)
    MAX_AREA_RATIO = 10        # Max ratio: detected_area / model_area
    MIN_AREA_RATIO = 0.15      # Min ratio: detected_area / model_area

    # === MULTI-INSTANCE DETECTION ===
    MAX_INSTANCES_PER_BOOK = 10  # Safety limit
    IOU_THRESHOLD = 0.3          # Overlap threshold for duplicate removal

    # === TEMPLATE VERIFICATION (Phase 2) ===
    # When Phase 1 finds at least one instance, Phase 2 uses the detected
    # scene region as a template to search for additional copies via NCC.
    TEMPLATE_NCC_THRESHOLD = 0.48  # NCC score threshold (lowered from 0.50
                                   # to catch the 4th copy in scene_10)
    TEMPLATE_SEARCH_MARGIN = 20    # Pixels of margin around detected region
    TEMPLATE_NMS_OVERLAP = 0.5     # Minimum template dimension fraction for NMS
    MAX_PHASE2_ADDITIONS = 4       # Maximum new detections from Phase 2 per model
    TEMPLATE_RELATIVE_SCORE = 0.6  # Phase 2 score must be >= 60% of best Phase 2 score


## 3. Data Classes

Structured representation of detection results.

In [3]:
@dataclass
class BoundingBox:
    """
    Represents a detected book instance.

    Stores the four corners of the bounding quadrilateral,
    which may not be axis-aligned due to the affine transformation.
    """
    top_left: Tuple[int, int]
    top_right: Tuple[int, int]
    bottom_right: Tuple[int, int]
    bottom_left: Tuple[int, int]
    area: int
    n_inliers: int
    inlier_ratio: float

    def get_polygon(self) -> np.ndarray:
        """Return corners as numpy array for geometric operations."""
        return np.array([self.top_left, self.top_right,
                        self.bottom_right, self.bottom_left], dtype=np.float32)

@dataclass
class BookDetection:
    """All detections for a single book model in a scene."""
    book_id: int
    model_path: str
    instances: List[BoundingBox]

## 4. RootSIFT Feature Extractor

### Why RootSIFT?

Standard SIFT descriptors are compared using Euclidean distance. However, SIFT descriptors are histograms of gradient orientations, and the **Hellinger kernel** (Bhattacharyya distance) is more appropriate for histogram comparison.

RootSIFT achieves this by:
1. L1-normalizing the SIFT descriptor
2. Taking the element-wise square root

This allows Euclidean distance to implicitly compute Hellinger distance, providing **~15-30% better matching accuracy**.

**Reference**: ArandjeloviÄ‡ & Zisserman, "Three things everyone should know to improve object retrieval" (CVPR 2012)

In [4]:
class RootSIFT:
    """
    RootSIFT feature extractor.

    Enhances SIFT descriptors for better matching performance
    by applying L1 normalization followed by square root.
    """

    def __init__(self):
        self.sift = cv2.SIFT_create(
            nfeatures=Config.SIFT_FEATURES,
            contrastThreshold=Config.SIFT_CONTRAST_THRESHOLD,
            edgeThreshold=Config.SIFT_EDGE_THRESHOLD
        )

    def detect_and_compute(self, image: np.ndarray):
        """
        Detect keypoints and compute RootSIFT descriptors.

        Args:
            image: Grayscale input image

        Returns:
            keypoints: List of cv2.KeyPoint
            descriptors: RootSIFT descriptors (Nx128 float32 array)
        """
        keypoints, descriptors = self.sift.detectAndCompute(image, None)

        if descriptors is None or len(descriptors) == 0:
            return keypoints, None

        # Convert to RootSIFT
        eps = 1e-7

        # Step 1: L1 normalize
        descriptors = descriptors / (np.sum(descriptors, axis=1, keepdims=True) + eps)

        # Step 2: Square root (Hellinger kernel)
        descriptors = np.sqrt(descriptors)

        return keypoints, descriptors.astype(np.float32)

## 5. Image Preprocessing

### Why CLAHE?

Shelf images have varying lighting conditions (shadows, reflections, uneven illumination). **CLAHE** (Contrast Limited Adaptive Histogram Equalization) normalizes local contrast while preventing over-amplification of noise.

### Parameter Choices

- **clipLimit = 2.0**: Moderate enhancement; higher values can amplify noise
- **tileGridSize = (4, 4)**: Finer grid provides better local adaptation for book spines

We use **fixed parameters** rather than adaptive selection because experiments showed that adaptive approaches (varying CLAHE based on image brightness) performed worse overall.

In [5]:
class ImagePreprocessor:
    """
    Preprocessing pipeline for lighting normalization.
    """

    def __init__(self):
        self.clahe = cv2.createCLAHE(
            clipLimit=Config.CLAHE_CLIP_LIMIT,
            tileGridSize=Config.CLAHE_GRID_SIZE
        )

    def preprocess(self, image: np.ndarray) -> np.ndarray:
        """
        Convert to grayscale and apply CLAHE.

        Args:
            image: BGR input image

        Returns:
            Preprocessed grayscale image
        """
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        if Config.CLAHE_FLAG:
            return self.clahe.apply(gray)
        else:
            return gray

## 6. Feature Matching with Combined Strategy

### Three Complementary Matching Approaches

We merge matches from three strategies via union (de-duplicated by `(queryIdx, trainIdx)` pair):

**Strategy 1: 5NN via FLANN**
For each model keypoint, find 5 nearest neighbors in the scene. Keep matches that are significantly better than the worst in their group. This allows one model keypoint to match multiple scene locations — essential for multi-instance detection.

**Strategy 2: BF 2NN Ratio Test**
Standard Lowe's ratio test (`best / second_best < 0.85`). Recovers matches that 5NN's group-based filtering misses, particularly for low-texture models.

**Strategy 3: BF 5NN Consecutive Ratio Test** *(inspired by Conti & Preda)*
For each model keypoint's 5 nearest neighbors, compare consecutive distances: keep `match[i]` if `match[i].distance < 0.6 * match[i+1].distance`. Unlike the standard ratio test that only checks rank 1 vs rank 2, this captures good matches at *any* rank position.

### Why merge all three?

Each strategy captures a different subset of correct matches. The union significantly improves recall — particularly critical for low-texture book spines where individual strategies may yield only 30-40 matches but the union reaches 80+.


In [6]:
class FeatureMatcher:
    """
    Combined feature matcher merging three complementary strategies.

    Strategies:
    1. 5NN via FLANN (group-based filtering)
    2. BF 2NN Lowe's ratio test
    3. BF 5NN consecutive ratio test (Conti & Preda)

    Matches are de-duplicated by (queryIdx, trainIdx) pair.
    """

    def __init__(self):
        # FLANN matcher for efficient approximate nearest neighbor search
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=100)
        self.flann = cv2.FlannBasedMatcher(index_params, search_params)
        # Brute-force matcher for exact nearest neighbor
        self.bf = cv2.BFMatcher()

    def match(self, des_model: np.ndarray, des_scene: np.ndarray,
              excluded_indices: Set[int] = None) -> List[cv2.DMatch]:
        """
        Match model descriptors to scene descriptors using combined strategy.

        Args:
            des_model: Model descriptors
            des_scene: Scene descriptors
            excluded_indices: Scene keypoint indices to exclude (already matched)

        Returns:
            List of good matches (union of all strategies, de-duplicated)
        """
        if des_model is None or des_scene is None:
            return []
        if len(des_model) < 2 or len(des_scene) < 5:
            return []

        if excluded_indices is None:
            excluded_indices = set()

        seen = set()       # Track (queryIdx, trainIdx) to avoid duplicates
        combined = []      # Final merged match list

        # --- Strategy 1: 5NN via FLANN ---
        # Allows one model keypoint to match multiple scene locations
        try:
            matches = self.flann.knnMatch(des_model, des_scene, k=5)
        except cv2.error:
            matches = []

        for match_group in matches:
            valid = [m for m in match_group if m.trainIdx not in excluded_indices]

            if len(valid) < 2:
                continue

            distances = [m.distance for m in valid]
            min_dist = distances[0]
            median_dist = distances[min(2, len(distances) - 1)]
            max_dist = distances[-1]

            if median_dist > 0 and min_dist > Config.DISTANCE_THRESHOLD * median_dist:
                continue

            threshold = max_dist / Config.OUTLIER_RATIO
            for m in valid:
                if m.distance <= threshold:
                    key = (m.queryIdx, m.trainIdx)
                    if key not in seen:
                        seen.add(key)
                        combined.append(m)

        # --- Strategy 2: BF 2NN with Lowe's ratio test ---
        try:
            bf_matches_2nn = self.bf.knnMatch(des_model, des_scene, k=2)
        except cv2.error:
            bf_matches_2nn = []

        for pair in bf_matches_2nn:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.trainIdx in excluded_indices:
                continue
            if m.distance < Config.BF_RATIO * n.distance:
                key = (m.queryIdx, m.trainIdx)
                if key not in seen:
                    seen.add(key)
                    combined.append(m)

        # --- Strategy 3: BF 5NN consecutive ratio test ---
        # Inspired by Conti & Preda: for each model keypoint's k=5 neighbors,
        # keep match[i] if its distance is significantly better than match[i+1].
        # This captures good matches at any rank, not just rank 1.
        try:
            bf_matches_5nn = self.bf.knnMatch(des_model, des_scene, k=5)
        except cv2.error:
            bf_matches_5nn = []

        for match_group in bf_matches_5nn:
            for i in range(len(match_group) - 1):
                m = match_group[i]
                m_next = match_group[i + 1]
                if m.trainIdx in excluded_indices:
                    continue
                if m.distance < Config.KNN_CONSECUTIVE_RATIO * m_next.distance:
                    key = (m.queryIdx, m.trainIdx)
                    if key not in seen:
                        seen.add(key)
                        combined.append(m)

        return combined


## 7. Geometry Utilities

Helper functions for geometric validation and IoU computation.

In [7]:
class GeometryUtils:
    """Geometric utility functions."""

    @staticmethod
    def polygon_area(points: np.ndarray) -> float:
        """Compute polygon area using Shoelace formula."""
        n = len(points)
        area = 0.0
        for i in range(n):
            j = (i + 1) % n
            area += points[i][0] * points[j][1]
            area -= points[j][0] * points[i][1]
        return abs(area) / 2.0

    @staticmethod
    def is_convex(points: np.ndarray) -> bool:
        """Check if quadrilateral is convex using cross product signs."""
        n = len(points)
        if n != 4:
            return False

        sign = None
        for i in range(n):
            p1, p2, p3 = points[i], points[(i+1)%n], points[(i+2)%n]
            cross = (p2[0]-p1[0]) * (p3[1]-p2[1]) - (p2[1]-p1[1]) * (p3[0]-p2[0])

            if abs(cross) < 1e-6:
                continue
            if sign is None:
                sign = cross > 0
            elif (cross > 0) != sign:
                return False
        return True

    @staticmethod
    def polygon_iou(poly1: np.ndarray, poly2: np.ndarray) -> float:
        """Compute Intersection over Union for two polygons."""
        # Find bounding region
        all_pts = np.vstack([poly1, poly2])
        x_min, y_min = np.floor(all_pts.min(axis=0)).astype(int) - 10
        x_max, y_max = np.ceil(all_pts.max(axis=0)).astype(int) + 10
        x_min, y_min = max(0, x_min), max(0, y_min)

        w, h = x_max - x_min, y_max - y_min
        if w <= 0 or h <= 0:
            return 0.0

        # Create masks
        offset = np.array([x_min, y_min])
        mask1 = np.zeros((h, w), dtype=np.uint8)
        mask2 = np.zeros((h, w), dtype=np.uint8)
        cv2.fillPoly(mask1, [(poly1 - offset).astype(np.int32)], 1)
        cv2.fillPoly(mask2, [(poly2 - offset).astype(np.int32)], 1)

        intersection = np.sum(mask1 & mask2)
        union = np.sum(mask1 | mask2)

        return intersection / union if union > 0 else 0.0

## 8. Similarity Transform Estimator

### Why Similarity Transform (Partial Affine) Instead of Full Affine or Homography?

Following the approach in [Conti & Preda, IPCV Assignment], we use `cv2.estimateAffinePartial2D` which estimates a **similarity transform** (4 DOF):

$$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} s \cos\theta & -s \sin\theta \\ s \sin\theta & s \cos\theta \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} + \begin{bmatrix} t_x \\ t_y \end{bmatrix}$$

This allows only:
- **Uniform scaling** (single scale factor $s$)
- **Rotation** (angle $\theta$)
- **Translation** ($t_x$, $t_y$)

It does **not** allow shear or non-uniform scaling.

### Why This Is Better for Books

- **Books preserve aspect ratio**: A book spine is always the same width-to-height ratio regardless of viewing angle
- **No shear in shelf images**: Books sit upright or lie flat, without trapezoidal distortion
- **Fewer parameters = more robust**: With only 4 unknowns (vs 6 for full affine, 8 for homography), RANSAC converges faster and is less likely to overfit to noise when we have limited matches (~8-15 per book instance)
- **Handles arbitrary rotation**: Unlike our previous validation that assumed upright books, the similarity transform naturally handles both vertical (0°) and horizontal (90°) book orientations

### Homography Conversion

The 2×3 affine matrix is converted to a 3×3 homography matrix by appending `[0, 0, 1]` as the third row. This allows using `cv2.perspectiveTransform` for corner projection, which is a standard OpenCV convention.

In [8]:
class AffineEstimator:
    """
    Similarity transform estimator using RANSAC.

    Uses cv2.estimateAffinePartial2D (4 DOF: uniform scale + rotation +
    translation) instead of full affine or homography. This provides the
    strongest inductive bias for book detection: books do not undergo
    shear or non-uniform scaling.

    The 2x3 matrix is converted to 3x3 homography format for convenient
    corner projection via cv2.perspectiveTransform.
    """

    def estimate(self, src_pts: np.ndarray, dst_pts: np.ndarray) -> Tuple[Optional[np.ndarray], Optional[np.ndarray], int]:
        """
        Estimate similarity transformation using RANSAC.

        Args:
            src_pts: Source points (model) - Nx2 array
            dst_pts: Destination points (scene) - Nx2 array

        Returns:
            (homography_matrix, inlier_mask, n_inliers)
            homography_matrix is 3x3 (similarity embedded in homography form)
        """
        if len(src_pts) < 2:
            return None, None, 0

        try:
            # estimateAffinePartial2D: 4 DOF similarity transform
            # (uniform scale + rotation + translation)
            M, inliers = cv2.estimateAffinePartial2D(
                src_pts.reshape(-1, 1, 2),
                dst_pts.reshape(-1, 1, 2),
                method=cv2.RANSAC,
                ransacReprojThreshold=Config.RANSAC_REPROJ_THRESHOLD,
                maxIters=2000,
                confidence=0.99
            )

            if M is None or inliers is None:
                return None, None, 0

            # Convert 2x3 affine to 3x3 homography matrix
            H = np.vstack([M, [0, 0, 1]])

            n_inliers = int(np.sum(inliers))
            return H, inliers, n_inliers

        except cv2.error:
            return None, None, 0

    def transform_corners(self, H: np.ndarray, model_shape: Tuple[int, int]) -> np.ndarray:
        """
        Project model corners to scene using the homography matrix.

        Args:
            H: 3x3 homography matrix
            model_shape: (height, width) of model image

        Returns:
            Projected corners as 4x2 array: [TL, TR, BR, BL]
        """
        h, w = model_shape
        corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
        projected = cv2.perspectiveTransform(corners, H)
        return projected.reshape(-1, 2)

    def get_rotation_deg(self, H: np.ndarray) -> float:
        """
        Extract rotation angle (degrees) from the similarity transform.

        For a similarity matrix:
            [s*cos(θ)  -s*sin(θ)  tx]
            [s*sin(θ)   s*cos(θ)  ty]
            [0          0          1]
        """
        return np.arctan2(H[1, 0], H[0, 0]) * 180 / np.pi

    def get_scale(self, H: np.ndarray) -> float:
        """
        Extract uniform scale factor from the similarity transform.
        """
        return np.sqrt(H[0, 0]**2 + H[1, 0]**2)

    def validate(self, H: np.ndarray, model_shape: Tuple[int, int],
                 scene_shape: Tuple[int, int]) -> bool:
        """
        Validate if the similarity transform produces a reasonable bounding box.

        Checks:
        1. Result is convex quadrilateral
        2. Projected corners are within scene bounds (with margin)
        3. Area is within reasonable bounds
        4. Scale factor is reasonable

        Note: We do NOT check corner orientation (top_left must be above
        bottom_left, etc.) because books can be horizontal in stacked shelves.
        The similarity transform naturally handles arbitrary rotation.
        """
        if H is None:
            return False

        # Project corners
        projected = self.transform_corners(H, model_shape)

        # === CHECK 1: Convexity ===
        if not GeometryUtils.is_convex(projected):
            return False

        # === CHECK 2: Within scene bounds ===
        scene_h, scene_w = scene_shape
        margin = max(scene_h, scene_w) * 0.2
        if (np.any(projected < -margin) or
            np.any(projected[:, 0] > scene_w + margin) or
            np.any(projected[:, 1] > scene_h + margin)):
            return False

        # === CHECK 3: Area bounds ===
        h, w = model_shape
        model_area = h * w
        detected_area = GeometryUtils.polygon_area(projected)

        if detected_area < Config.MIN_AREA:
            return False

        ratio = detected_area / model_area
        if ratio < Config.MIN_AREA_RATIO or ratio > Config.MAX_AREA_RATIO:
            return False

        # === CHECK 4: Reasonable scale ===
        # For similarity transform, scale is uniform, so we just check
        # it's within a reasonable range
        scale = self.get_scale(H)
        if scale < 0.1 or scale > 5.0:
            return False

        return True

## 9. Book Detector

### Multi-Instance Detection: Two-Phase Approach

#### Phase 1: Iterative Feature-Based Detection with Keypoint Exclusion

1. Match all model keypoints to scene
2. Estimate similarity transformation (RANSAC)
3. If valid: save detection, add inlier scene keypoint indices to exclusion set
4. Re-match with excluded indices, repeat until no valid detections remain

This approach is fast (features extracted once) and allows multiple RANSAC
attempts even when match quality degrades, unlike scene masking which stops
after the first failure.

#### Phase 2: Orientation-Aware Template Search (NCC)

For books with limited texture, Phase 1 may find only one instance.
Phase 2 uses the first detection as a template and searches the **full scene**:
- Detects **orientation** (vertical vs horizontal) from the similarity transform rotation
- Applies NMS along the **correct axis** (X for side-by-side, Y for stacked)

### Why Similarity Transform?

We use `cv2.estimateAffinePartial2D` (4 DOF: uniform scale + rotation + translation)
rather than full affine (6 DOF) or homography (8 DOF). Books don't undergo shear
or non-uniform scaling, so this provides the strongest inductive bias.


In [9]:
class BookDetector:
    """
    Main book detection pipeline.

    Pipeline:
    1. Preprocess images (CLAHE)
    2. Extract RootSIFT features
    3. Match features (combined 5NN + BF + consecutive ratio)
    4. Phase 1: Iterative detection with keypoint exclusion
    5. Phase 2: Orientation-aware template search (NCC)
    """

    def __init__(self):
        self.preprocessor = ImagePreprocessor()
        self.feature_extractor = RootSIFT()
        self.matcher = FeatureMatcher()
        self.affine_estimator = AffineEstimator()
        self.model_cache = {}  # Cache model features

    def load_model(self, model_path: str):
        """Load and cache model image features."""
        if model_path in self.model_cache:
            return self.model_cache[model_path]

        img = cv2.imread(model_path)
        if img is None:
            raise ValueError(f"Could not load: {model_path}")

        gray = self.preprocessor.preprocess(img)
        kp, des = self.feature_extractor.detect_and_compute(gray)

        self.model_cache[model_path] = (img, kp, des)
        return img, kp, des

    def detect_in_scene(self, scene_path: str, model_paths: List[str],
                        verbose: bool = False) -> Tuple[List[BookDetection], np.ndarray]:
        """
        Detect all books in a scene image.
        """
        scene_img = cv2.imread(scene_path)
        if scene_img is None:
            raise ValueError(f"Could not load: {scene_path}")

        scene_gray = self.preprocessor.preprocess(scene_img)
        scene_kp, scene_des = self.feature_extractor.detect_and_compute(scene_gray)
        scene_gray_raw = cv2.cvtColor(scene_img, cv2.COLOR_BGR2GRAY)

        if verbose:
            print(f"Scene: {Path(scene_path).name} - {len(scene_kp)} keypoints")

        detections = []
        result_img = scene_img.copy()

        for book_id, model_path in enumerate(model_paths):
            model_img, model_kp, model_des = self.load_model(model_path)

            if model_des is None or len(model_kp) < Config.MIN_MATCH_COUNT:
                detections.append(BookDetection(book_id, model_path, []))
                continue

            instances = self._detect_instances(
                model_img, model_kp, model_des,
                scene_img, scene_kp, scene_des,
                scene_gray_raw
            )

            detection = BookDetection(book_id, model_path, instances)
            detections.append(detection)
            self._draw_detection(result_img, detection)

            if verbose and len(instances) > 0:
                print(f"  Book {book_id}: {len(instances)} instance(s)")

        return detections, result_img

    def _detect_instances(self, model_img, model_kp, model_des,
                          scene_img, scene_kp, scene_des,
                          scene_gray_raw) -> List[BoundingBox]:
        """
        Detect all instances using two-phase approach.

        Phase 1: Iterative similarity estimation + keypoint exclusion.
        Phase 2: Orientation-aware NCC template search.
        """
        instances = []
        instance_transforms = []
        excluded_indices: Set[int] = set()

        for _ in range(Config.MAX_INSTANCES_PER_BOOK):
            # Match with exclusion of previously used keypoints
            matches = self.matcher.match(model_des, scene_des, excluded_indices)

            if len(matches) < Config.MIN_MATCH_COUNT:
                break

            src_pts = np.float32([model_kp[m.queryIdx].pt for m in matches])
            dst_pts = np.float32([scene_kp[m.trainIdx].pt for m in matches])
            match_indices = [m.trainIdx for m in matches]

            # Estimate similarity transformation
            H, inlier_mask, n_inliers = self.affine_estimator.estimate(src_pts, dst_pts)

            if H is None:
                break

            inlier_ratio = n_inliers / len(matches)
            if n_inliers < Config.MIN_INLIERS or inlier_ratio < Config.MIN_INLIERS_RATIO:
                # Exclude these inliers and try again
                inlier_indices = [match_indices[i] for i in range(len(match_indices))
                                  if inlier_mask[i]]
                excluded_indices.update(inlier_indices)
                continue

            if not self.affine_estimator.validate(H, model_img.shape[:2], scene_img.shape[:2]):
                inlier_indices = [match_indices[i] for i in range(len(match_indices))
                                  if inlier_mask[i]]
                excluded_indices.update(inlier_indices)
                continue

            # Create bounding box
            projected = self.affine_estimator.transform_corners(H, model_img.shape[:2])
            area = GeometryUtils.polygon_area(projected)

            bbox = BoundingBox(
                top_left=tuple(map(int, projected[0])),
                top_right=tuple(map(int, projected[1])),
                bottom_right=tuple(map(int, projected[2])),
                bottom_left=tuple(map(int, projected[3])),
                area=int(area),
                n_inliers=n_inliers,
                inlier_ratio=inlier_ratio
            )

            # Check for duplicates (NMS)
            is_duplicate = any(
                GeometryUtils.polygon_iou(bbox.get_polygon(), ex.get_polygon())
                > Config.IOU_THRESHOLD
                for ex in instances
            )

            if is_duplicate:
                inlier_indices = [match_indices[i] for i in range(len(match_indices))
                                  if inlier_mask[i]]
                excluded_indices.update(inlier_indices)
                continue

            # Valid new instance!
            instances.append(bbox)
            instance_transforms.append(H)

            # Exclude inlier keypoints for next iteration
            inlier_indices = [match_indices[i] for i in range(len(match_indices))
                              if inlier_mask[i]]
            excluded_indices.update(inlier_indices)

        # === PHASE 2: Orientation-aware template search ===
        if len(instances) >= 1 and len(instance_transforms) >= 1:
            instances = self._template_search(
                instances, instance_transforms[0],
                scene_img, scene_gray_raw
            )

        return instances

    def _template_search(self, instances: List[BoundingBox],
                         H: np.ndarray,
                         scene_img: np.ndarray,
                         scene_gray_raw: np.ndarray) -> List[BoundingBox]:
        """
        Phase 2: Orientation-aware template search using NCC.

        Uses the same similarity transform from Phase 1. For each NCC peak, we
        compute the translation offset from the reference detection and apply
        the same rotation/scale to the model corners.

        This ensures Phase 2 boxes have the same quality as Phase 1 boxes.
        """
        ref = instances[0]
        ref_poly = ref.get_polygon()

        rotation_deg = self.affine_estimator.get_rotation_deg(H)
        is_horizontal = 45 < abs(rotation_deg) < 135

        scene_h, scene_w = scene_img.shape[:2]
        x_min = max(0, int(np.min(ref_poly[:, 0])))
        x_max = min(scene_w, int(np.max(ref_poly[:, 0])))
        y_min = max(0, int(np.min(ref_poly[:, 1])))
        y_max = min(scene_h, int(np.max(ref_poly[:, 1])))

        template = scene_gray_raw[y_min:y_max, x_min:x_max]
        t_h, t_w = template.shape[:2]

        if t_w < 5 or t_h < 5:
            return instances

        if scene_gray_raw.shape[0] <= t_h or scene_gray_raw.shape[1] <= t_w:
            return instances

        result = cv2.matchTemplate(
            scene_gray_raw, template, cv2.TM_CCOEFF_NORMED
        )

        # NMS distance along the separation axis
        if is_horizontal:
            nms_min_dist = t_h * Config.TEMPLATE_NMS_OVERLAP
        else:
            nms_min_dist = t_w * Config.TEMPLATE_NMS_OVERLAP

        # Find local maxima above threshold
        peaks = []
        for x in range(result.shape[1]):
            for y in range(result.shape[0]):
                if result[y, x] >= Config.TEMPLATE_NCC_THRESHOLD:
                    is_max = True
                    for dx in range(-5, 6):
                        for dy in range(-3, 4):
                            nx, ny = x + dx, y + dy
                            if (0 <= nx < result.shape[1] and
                                0 <= ny < result.shape[0]):
                                if result[ny, nx] > result[y, x]:
                                    is_max = False
                                    break
                        if not is_max:
                            break
                    if is_max:
                        peaks.append((x, y, result[y, x]))

        # NMS along correct axis
        peaks.sort(key=lambda p: -p[2])
        kept_peaks = []
        for px, py, score in peaks:
            is_dup = False
            for kx, ky, _ in kept_peaks:
                if is_horizontal:
                    if abs(py - ky) < nms_min_dist:
                        is_dup = True
                        break
                else:
                    if abs(px - kx) < nms_min_dist:
                        is_dup = True
                        break
            if not is_dup:
                kept_peaks.append((px, py, score))

        # Reference detection's axis-aligned bounding box center
        ref_center_x = (x_min + x_max) / 2.0
        ref_center_y = (y_min + y_max) / 2.0

        # Reference detection's ROTATED polygon center
        ref_poly_center = np.mean(ref_poly, axis=0)

        # Convert peaks to ROTATED bounding boxes by shifting the
        # Phase 1 polygon. Each NCC peak gives us the top-left corner
        # of the axis-aligned bounding box of the match. We compute
        # the offset from the reference's axis-aligned box and apply
        # it to the rotated polygon.
        for px, py, score in kept_peaks:
            peak_center_x = px + t_w / 2.0
            peak_center_y = py + t_h / 2.0

            # Offset from reference (in axis-aligned coords)
            dx = peak_center_x - ref_center_x
            dy = peak_center_y - ref_center_y

            # Shift the rotated polygon by the same offset
            shifted_poly = ref_poly + np.array([dx, dy])

            area = GeometryUtils.polygon_area(shifted_poly)
            bbox = BoundingBox(
                top_left=tuple(map(int, shifted_poly[0])),
                top_right=tuple(map(int, shifted_poly[1])),
                bottom_right=tuple(map(int, shifted_poly[2])),
                bottom_left=tuple(map(int, shifted_poly[3])),
                area=int(area),
                n_inliers=0,
                inlier_ratio=score
            )

            # Reject if overlaps with existing detection
            is_duplicate = any(
                GeometryUtils.polygon_iou(
                    bbox.get_polygon(), ex.get_polygon()
                ) > Config.IOU_THRESHOLD
                for ex in instances
            )
            if not is_duplicate:
                instances.append(bbox)

        return instances

    def _draw_detection(self, img: np.ndarray, detection: BookDetection):
        """Draw bounding boxes on image."""
        colors = [
            (0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0),
            (255, 0, 255), (0, 255, 255), (128, 0, 255), (255, 128, 0)
        ]
        color = colors[detection.book_id % len(colors)]

        for inst in detection.instances:
            pts = np.array([inst.top_left, inst.top_right,
                           inst.bottom_right, inst.bottom_left], dtype=np.int32)
            cv2.polylines(img, [pts], True, color, 3)

            label = f"Book {detection.book_id}"
            pos = (inst.top_left[0], max(inst.top_left[1] - 10, 20))
            cv2.putText(img, label, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


## 10. Output Formatting

In [10]:
def format_output(detections: List[BookDetection]) -> str:
    """
    Format detection results as specified in the assignment.

    Output format:
    Book X - N instance(s) found:
      Instance 1 {top_left: (x,y), top_right: (x,y), ...}
    """
    lines = []

    for det in detections:
        if len(det.instances) > 0:
            lines.append(f"Book {det.book_id} - {len(det.instances)} instance(s) found:")

            for i, inst in enumerate(det.instances, 1):
                lines.append(
                    f"  Instance {i} {{"
                    f"top_left: {inst.top_left}, "
                    f"top_right: {inst.top_right}, "
                    f"bottom_left: {inst.bottom_left}, "
                    f"bottom_right: {inst.bottom_right}, "
                    f"area: {inst.area}px}}"
                )

    return "\n".join(lines)

## 11. Load Dataset

In [11]:

import urllib.request
import zipfile
import os
import shutil

# --- Configuration ---
# Replace this with the exact full 40-character commit hash you want to pin
COMMIT_HASH = "a455e8a87eb21bd2c649a20eb22e3273b025d185"

# GitHub automatically formats commit download URLs and folder names like this:
repo_zip_url = f"https://github.com/niccolozanotti/ipcv-assignments/archive/{COMMIT_HASH}.zip"
zip_path = "temp_repo.zip"
extract_dir = f"ipcv-assignments-{COMMIT_HASH}"

# 1. Download the specific commit zip
print(f"Downloading repository at commit {COMMIT_HASH[:7]}...")
urllib.request.urlretrieve(repo_zip_url, zip_path)

# 2. Extract ONLY the dataset folder
print("Extracting dataset folder...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        # Look specifically for the dataset folder inside the unzipped directory
        if file.startswith(f"{extract_dir}/dataset/"):
            zip_ref.extract(file, path=".")

# 3. Move the dataset folder to the current directory and clean up
if os.path.exists("dataset"):
    shutil.rmtree("dataset") # Clean up if you are re-running the cell

shutil.move(f"{extract_dir}/dataset", "dataset")
shutil.rmtree(extract_dir)
os.remove(zip_path)

print("Dataset successfully downloaded and ready to use!")

from pathlib import Path

# Updated paths pointing to the locally downloaded folder
MODELS_PATH = "dataset/models"
SCENES_PATH = "dataset/scenes"

# Load file lists
# The sorting logic stays exactly the same to ensure model_10 comes after model_9
model_files = sorted(Path(MODELS_PATH).glob("model_*.png"), key=lambda x: int(x.stem.split('_')[1]))
scene_files = sorted(Path(SCENES_PATH).glob("scene_*.jpg"), key=lambda x: int(x.stem.split('_')[1]))

model_paths = [str(f) for f in model_files]
scene_paths = [str(f) for f in scene_files]

print(f"Found {len(model_paths)} models and {len(scene_paths)} scenes")

Extracting dataset folder...
Dataset successfully downloaded and ready to use!
Found 22 models and 29 scenes


## 12. Run Detection

In [ ]:
# Initialize detector
detector = BookDetector()

# Process all scenes
all_results = {}
all_images = {}

for idx, scene_path in enumerate(scene_paths):
    print(f"\nProcessing scene {idx}: {Path(scene_path).name}")

    detections, result_img = detector.detect_in_scene(scene_path, model_paths, verbose=False)

    all_results[idx] = detections
    all_images[idx] = result_img

    total = sum(len(d.instances) for d in detections)
    books = sum(1 for d in detections if len(d.instances) > 0)
    print(f"  Found {total} instances of {books} books")

print("\n" + "="*60)
print("PROCESSING COMPLETE")
print("="*60)


Processing scene 0: scene_0.jpg
  Found 0 instances of 0 books

Processing scene 1: scene_1.jpg
  Found 2 instances of 1 books

Processing scene 2: scene_2.jpg
  Found 2 instances of 1 books

Processing scene 3: scene_3.jpg
  Found 11 instances of 1 books

Processing scene 4: scene_4.jpg
  Found 4 instances of 2 books

Processing scene 5: scene_5.jpg
  Found 1 instances of 1 books

Processing scene 6: scene_6.jpg
  Found 1 instances of 1 books

Processing scene 7: scene_7.jpg
  Found 2 instances of 1 books

Processing scene 8: scene_8.jpg
  Found 0 instances of 0 books

Processing scene 9: scene_9.jpg
  Found 4 instances of 1 books

Processing scene 10: scene_10.jpg


## 13. Results

In [ ]:
# Print formatted results
for idx, detections in all_results.items():
    detected = [d for d in detections if len(d.instances) > 0]
    if detected:
        print(f"\n{'='*60}")
        print(f"SCENE: {Path(scene_paths[idx]).name}")
        print(f"{'='*60}")
        print(format_output(detected))

## 14. Visualization

In [ ]:
# Visualize all results
for idx in range(len(scene_paths)):
    if idx not in all_images:
        continue

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(all_images[idx], cv2.COLOR_BGR2RGB))
    plt.title(f"Scene {idx}: {Path(scene_paths[idx]).name}")
    plt.axis('off')
    plt.show()

## Full Pipeline Diagnostic

The function below traces **every intermediate step** when detecting a specific book in a specific scene.
It reports:
- Preprocessing and feature extraction statistics
- Match counts from both 5NN and BF strategies (with overlap analysis)
- Each RANSAC iteration: affine matrix, scale/rotation, all 7 validation checks
- Phase 2 template matching: NCC scores, peaks, NMS, duplicate rejection
- Final detection count and visualization

Usage:
```python
diagnose_full(detector, scene_paths, model_paths, scene_idx=10, book_id=19)
```

## 15. Statistics

In [ ]:
# Compute statistics
total_detections = 0
detections_per_scene = []
detections_per_book = defaultdict(int)

for detections in all_results.values():
    scene_total = sum(len(d.instances) for d in detections)
    detections_per_scene.append(scene_total)
    total_detections += scene_total

    for d in detections:
        if len(d.instances) > 0:
            detections_per_book[d.book_id] += len(d.instances)

print("DETECTION STATISTICS")
print("="*60)
print(f"Total detections: {total_detections}")
print(f"Average per scene: {np.mean(detections_per_scene):.2f}")
print(f"Max in one scene: {max(detections_per_scene)}")
print(f"\nUnique books detected: {len(detections_per_book)}")

print("\nTop 10 most detected books:")
for book_id, count in sorted(detections_per_book.items(), key=lambda x: -x[1])[:10]:
    print(f"  Book {book_id}: {count} instances")

## 16. Conclusion

### Summary

This pipeline implements book detection using traditional computer vision:

1. **RootSIFT features** for robust descriptor matching
2. **Three-strategy matching** (5NN + BF ratio + consecutive ratio) for maximum recall
3. **Similarity transform** (4 DOF via `estimateAffinePartial2D`) for geometric verification
4. **Iterative keypoint exclusion** for multi-instance detection
5. **Orientation-aware template search** (NCC) for recovering copies with limited features

### Key Design Decisions

| Decision | Rationale |
|----------|-----------|
| Similarity (4 DOF) over homography (8 DOF) | Books don't undergo perspective distortion; stronger inductive bias |
| Keypoint exclusion over scene masking | Allows multiple RANSAC attempts even with degraded match quality |
| Full-scene template search | Handles both vertical and horizontal book arrangements |
| NMS along orientation axis | Prevents collapsing stacked/adjacent copies |
| Three matching strategies | Each captures different correct matches; union improves recall |

### Limitations

- May struggle with heavily occluded books
- Performance depends on book cover texture (low-texture covers yield fewer features)
- Phase 2 template matching produces axis-aligned boxes (not rotated to match book orientation)
- FLANN non-determinism can cause slight variation between runs
